# BIGRU model met embeddings
## Geen embedding pre-training

In [1]:
from torch import optim
from data.dataloader import DatasetLoader
from torch.utils.data import DataLoader
import torch
from models.BiGRU import BiGRU
from torch import nn
from tqdm import tqdm
import numpy as np
import pandas as pd

## 1. Dataset uitlezen

In [6]:
dataset = DatasetLoader("eventlog/phone_usage_cleaned.csv", seq_length=10, max_apps=200)
removeapps = ["Screen on (locked)",
              "Screen off (locked)",
              "Screen on (unlocked)",
              "Screen off",
              "Samsung Experience Service",
              "Package installer",
              "System UI",
              "Customisation Service",
              "Configuration update",
              "EmergencyManagerService",
              "DeviceKeystring",
              "Samsung Keyboard",
              "HwModuleTest",
              "Device shutdown",
              "Device boot"]
dataset.clean(removeapps)
train_dataloader = DataLoader(dataset)

## 2. bigru model aanmaken en trainen

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
bigru = BiGRU(max_apps=200,
              seq_length=10,
              n_gru=64)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(bigru.parameters(), lr=0.0015)
# bigru.src_embedding.weight = torch.nn.Parameter(torch.zeros(size=(200, 4)))
bigru.to(device=device)

running_loss = 0.0
running_accuracy_list = []
i = 0
total = 0
correct = 0
epochs = 4
for prev_apps_indices, curr_app_index, curr_app_onehot in tqdm(train_dataloader):
    prev_apps_indices, curr_app_index = prev_apps_indices.to(device), curr_app_index.to(device)

    for j in range(epochs):
        optimizer.zero_grad()

        outputs = bigru(prev_apps_indices[0])

        loss = criterion(outputs, curr_app_index)
        loss.backward()
        optimizer.step()

        total += 1
        _, predicted = torch.topk(outputs.data, 5)
        correct += (curr_app_index in predicted)*1

        running_loss += loss.item()
    if i % 2000 == 1999:
        print('loss: %.3f' %
              (running_loss / 2000))
        running_loss = 0.0
        print('Accuracy of the network is:' + str(100 * correct / total))
        running_accuracy_list.append(100 * correct / total)
        total = 0
        correct = 0
    i += 1

100%|██████████| 33627/33627 [05:01<00:00, 111.67it/s]


loss: 10.681
Accuracy of the network is:71.1125
loss: 11.427
Accuracy of the network is:64.325
loss: 11.692
Accuracy of the network is:64.4625
loss: 11.526
Accuracy of the network is:63.4625
loss: 9.788
Accuracy of the network is:75.4
loss: 10.694
Accuracy of the network is:73.675
loss: 10.246
Accuracy of the network is:72.7625
loss: 10.649
Accuracy of the network is:74.725
loss: 9.870
Accuracy of the network is:77.95
loss: 11.660
Accuracy of the network is:73.4375
loss: 10.995
Accuracy of the network is:75.5125
loss: 12.050
Accuracy of the network is:70.7625
loss: 12.536
Accuracy of the network is:64.8
loss: 10.766
Accuracy of the network is:73.6875
loss: 9.580
Accuracy of the network is:78.0625
loss: 10.400
Accuracy of the network is:74.0625


## 3. Running accuracy opslaan

In [35]:
AccuracyComparison = pd.read_csv("outputdata/accuracycomparison.csv", index_col=False)
AccuracyComparison["Embeddings"] = running_accuracy_list
AccuracyComparison.to_csv("outputdata/accuracycomparison.csv", index=False)
